In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
fashion_mnist = keras.datasets.fashion_mnist

(train_img, train_labels), (test_img, test_labels) = fashion_mnist.load_data()

train_img = train_img / 255.0
test_img = test_img / 255.0

In [4]:
train_labels.shape

(60000,)

In [5]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax')) # 10개 분류!

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# InternalError:  Blas GEMM launch failed : a.shape=(32, 784), b.shape=(784, 128), m=32, n=128, k=784

돌아가고 있는 주피터 노트북 종료하고 실행하면 해결된다.

In [8]:
model.fit(train_img, train_labels) 

Train on 60000 samples
60000/60000 [==============================] - 4s 69us/sample - loss: 0.5029 - accuracy: 0.8228


# 원핫 인코딩

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
import sklearn

sklearn.__version__

'0.21.3'

In [11]:
ohe = OneHotEncoder()

In [12]:
ohe.fit_transform(train_labels)

ValueError: Expected 2D array, got 1D array instead:
array=[9 0 0 ... 3 0 5].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

위와 같은 에러가 나오면 reshape을 해서, shape를 맞춰주면 된다.

In [13]:
ohe.fit_transform(train_labels.reshape(-1, 1)).toarray() # 넘파이 포맷으로 바꿔준다.

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [14]:
from tensorflow.keras.utils import to_categorical

In [15]:
to_categorical(test_labels) # 이렇게 편하게 바꿔주기도 한다... 이게 훨씬 더 편하네.

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

판다스 겟더미스도 원핫지원함.. 포스팅할까? ㅋ

### 라벨 원핫으로 만들기

In [16]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [17]:
model3 = keras.Sequential()
model3.add(keras.layers.Flatten(input_shape = (28, 28)))
model3.add(keras.layers.Dense(128, activation = 'relu'))
model3.add(keras.layers.Dense(10, activation = 'softmax'))

In [18]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [19]:
# sparse는 내부적으로 원핫을 해준다.
model3.compile(loss = 'categorical_crossentropy', # 원핫을 쓸때는 sparse를 해주면 안된다.
               optimizer='adam',
               metrics = ['acc'])

In [20]:
ff = model3.fit(train_img, train_labels, epochs = 10, verbose = 1)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 4s 61us/sample - loss: 0.4977 - acc: 0.8251
Epoch 2/10
60000/60000 [==============================] - 4s 59us/sample - loss: 0.3796 - acc: 0.8637
Epoch 3/10
60000/60000 [==============================] - 3s 57us/sample - loss: 0.3397 - acc: 0.8763
Epoch 4/10
60000/60000 [==============================] - 4s 59us/sample - loss: 0.3140 - acc: 0.8849
Epoch 5/10
60000/60000 [==============================] - 4s 59us/sample - loss: 0.2955 - acc: 0.8912
Epoch 6/10
60000/60000 [==============================] - 4s 59us/sample - loss: 0.2802 - acc: 0.8976
Epoch 7/10
60000/60000 [==============================] - 3s 58us/sample - loss: 0.2679 - acc: 0.9016
Epoch 8/10
60000/60000 [==============================] - 3s 58us/sample - loss: 0.2581 - acc: 0.9037
Epoch 9/10
60000/60000 [==============================] - 4s 60us/sample - loss: 0.2505 - acc: 0.9072
Epoch 10/10
60000/60000 [==============================] - 

시간을 측정해보면 원핫 인코딩하는 애가 더 효율적이긴 한데... 차원이 더 커진다. 학습하는데 걸리는 시간이 더 걸릴 수도 있다.

라벨데이터가 있으면, 오히려 원핫 인코딩 안하는게 좋다. 왜냐면 라벨을 펼쳐서 일일이 찾아야 한다.

라벨 인코딩 vs 원핫 인코딩

우리는 라벨 인코딩만 쓸 거다. 우리는 y값이 원핫인지 아닌지는 체크정도 해줘야 한다. -> 웟핫이면 loss에서 sparse를 붙인다.

성능은 라벨 인코딩과 원핫 인코딩 모두 똑같다.

---

사이킷런의 inverse_transform -> 원핫에서 어떤 라벨인지 거꾸로 변환해준다.

# Flatten 안쓰기

데이터 자체를 바꿔서 들어가면 된다.

★★★ 실무에서는 flatten 안쓴다. 데이터에서 flatten 시켜버리면 학습이 더 빠르게 되니까.

In [27]:
fashion_mnist = keras.datasets.fashion_mnist

(train_img, train_labels), (test_img, test_labels) = fashion_mnist.load_data()

train_img = train_img / 255.0
test_img = test_img / 255.0

train_img2 = train_img.reshape(-1, 28*28)
test_img2 = train_img.reshape(-1, 28*28)

In [28]:
model4 = keras.Sequential()
model4.add(keras.layers.Dense(128, input_shape = (784,), activation = 'relu'))
model4.add(keras.layers.Dense(10, activation = 'softmax')) # output shape 맞춰야 한다.

In [29]:
model4.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_7 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [30]:
# sparse는 내부적으로 원핫을 해준다.
model4.compile(loss = 'sparse_categorical_crossentropy', # 원핫을 쓸때는 sparse를 해주면 안된다.
               optimizer='adam',
               metrics = ['acc'])

In [31]:
model4.fit(train_img2, train_labels)

Train on 60000 samples
60000/60000 [==============================] - 4s 63us/sample - loss: 0.4979 - acc: 0.8267
